# T3: Analize de Dados do SiaSUS

Forma: Trabalho em grupo com apresentação do ambiente e do código para o professor.

Objetivo:
Nesta primeira parte, que deve estar pronta para a aula do dia 03/11/2025, os grupos devem obter, preparar e carregar em um banco de dados relacional os dados do SIASUS (Sistema de Informações Ambulatoriais do SUS), disponibilizados publicamente pelo DataSUS.

Instruções:
1. 1) Obtenção da base de dados
 - Acesse o repositório do DataSUS e faça o download dos arquivos do SIASUS no formato .dbc.
 - Selecione os arquivos referentes a procedimentos ambulatoriais (ex.: PARS e outros complementares necessários para a descrição da tabela PARS).
2. 2) Conversão de formato
 - Converta os arquivos .dbc para .dbf utilizando as ferramentas recomendadas (ex.: TABWIN).
 - Certifique-se de validar a integridade dos arquivos após a conversão.
3) Visualização dos dados
 - Utilize o TABWIN, software oficial do DataSUS, para visualizar os arquivos .dbf.
 - A partir do TABWIN, gere:
  - Scripts SQL de criação de tabelas.
4) Carga dos dados em SGBD:
 - Insira os dados no banco de dados relacional de sua escolha (ex. MySQL, PostgreSQL, IBM DB2, etc).
 - Para arquivos menores, utilize diretamente os scripts de inserção gerados pelo TABWIN.
 - Para arquivos muito grandes (como o PARS, que contém milhões de registros), gere um arquivo CSV a partir do .dbf e utilize um comando de LOAD (carga em lote) no SGBD escolhido.
 - Se optar pelo MySQL (que foi o utilizado em aula):
  - Pesquise e utilize a instrução LOAD DATA na linha de comando do servidor (para evitar problemas de permissões com o cliente Workbench) conforme a documentação oficial: https://dev.mysql.com/doc/refman/8.4/en/load-data.html
Exemplo de comando LOAD DATA no terminal do servidor: LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/PARS2508.csv' INTO TABLE pars CHARACTER SET latin1 FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES  TERMINATED BY '\r\n' IGNORE 1 ROWS; 
5) A continuidade do trabalho, na aula do dia 03/11/2025, depende do ambiente do banco de dados funcionando.

## Principais libs que usaremos: 

In [ ]:
# Esse projeto foi feito com o objetivo educativo para a matéria de Programção para Ciência dos dados
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# /kaggle/input/sus-data-csv/CADGERRS.csv

# Declarando arquivos:

In [ ]:
import pandas as pd

# Define o caminho base para facilitar (opcional, mas limpo)
base_path = '/kaggle/input/'

# --- 1. TAREFAS FATO (Os eventos, a produção) ---
# Estas são as tabelas principais. Elas registram os eventos (atendimentos).
# Os nomes (PARS2501, PARS2505, PARS2508) sugerem que são dados mensais
# (ex: Janeiro, Maio, Agosto de 2025). Vamos carregá-los e juntá-los.

print("Carregando tabelas de produção (PARS)...")

# O que é: Tabela FATO - Produção Ambulatorial (Parte 1, ex: Mês 1)
# O que vamos contar: Registros de procedimentos, valores, local, paciente, etc.
df_prod_01 = pd.read_csv(base_path + 'PARS2501.csv', encoding='latin1', low_memory=False)

# O que é: Tabela FATO - Produção Ambulatorial (Parte 2, ex: Mês 5)
# O que vamos contar: Mais registros de procedimentos...
df_prod_05 = pd.read_csv(base_path + 'PARS2505.csv', encoding='latin1', low_memory=False)

# O que é: Tabela FATO - Produção Ambulatorial (Parte 3, ex: Mês 8)
# O que vamos contar: Mais registros de procedimentos...
df_prod_08 = pd.read_csv(base_path + 'PARS2508.csv', encoding='latin1', low_memory=False)

# --- 2. TAREFAS DIMENSÃO (Os dicionários, "de-para") ---
# Estas são as tabelas auxiliares que dão nome aos códigos das tabelas FATO.

print("Carregando tabelas de dimensão (dicionários)...")

# O que é: Tabela de Procedimentos (SIGTAP)
# O que vamos usar: Dar nome aos códigos de procedimento (ex: '0301010072' -> 'CONSULTA MEDICA')
df_procedimentos = pd.read_csv(base_path + 'TB_SIGTAW.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Municípios (IBGE)
# O que vamos usar: Dar nome aos códigos de município (ex: '431020' -> 'Ijuí').
# ESSENCIAL para a análise de fluxo de pacientes (item 4 do seu trabalho).
df_municipios = pd.read_csv(base_path + 'tb_municip.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Doenças (Classificação Internacional de Doenças - CID)
# O que vamos usar: Dar nome aos códigos de diagnóstico (ex: 'I10' -> 'Hipertensão Essencial').
df_cid = pd.read_csv(base_path + 'S_CID.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Ocupações (Classificação Brasileira de Ocupações - CBO)
# O que vamos usar: Dar nome aos códigos de ocupação do profissional
# (ex: '225125' -> 'Médico Clínico').
df_cbo = pd.read_csv(base_path + 'CBO.csv', encoding='latin1', low_memory=False)

# O que é: Cadastro de Estabelecimentos (CNES)
# O que vamos usar: Dar nome e endereço aos códigos de hospitais, clínicas e UBS
# (ex: '2254611' -> 'HOSPITAL DE CARIDADE DE IJUI').
df_estabelecimentos = pd.read_csv(base_path + 'CADGERRS.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Relação (RL) Município -> Microrregião
# O que vamos usar: Ligar um código de município a um código de microrregião.
df_rl_mun_micro = pd.read_csv(base_path + 'rl_municip_micibge.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Microrregiões (IBGE)
# O que vamos usar: Dar nome ao código da microrregião (ex: '11001' -> 'Porto Velho').
df_microrregioes = pd.read_csv(base_path + 'tb_micibge.csv', encoding='latin1', low_memory=False)

print("\nTodos os arquivos foram carregados com sucesso!")

In [ ]:
# Junta as 3 tabelas de produção (uma em cima da outra)
df_producao_total = pd.concat([df_prod_01, df_prod_05, df_prod_08], ignore_index=True)

# Libera a memória, já que não precisamos mais das partes separadas
del df_prod_01
del df_prod_05
del df_prod_08

print(f"Tabelas de produção combinadas!")
print(f"Total de registros para analisar: {len(df_producao_total)}")



In [ ]:
df_producao_total.head(n=10)

In [ ]:
print(df_producao_total.columns.tolist())

In [ ]:
# A chave da nossa tabela principal (produção) é 'PA_MUNPCN'
chave_principal = 'PA_MUNPCN'

# A chave da nossa tabela auxiliar (municípios) é 'CO_MUNICIP'
chave_auxiliar = 'CO_MUNICIP'

# Converte ambas as colunas-chave para string (texto) para garantir o merge
df_producao_total[chave_principal] = df_producao_total[chave_principal].astype(str)
df_municipios[chave_auxiliar] = df_municipios[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

In [ ]:
# Vamos criar nosso dataframe de análise final, começando com este merge
# Puxamos apenas as colunas de nome e UF da tabela de municípios
df_analise = pd.merge(
    left=df_producao_total,
    right=df_municipios[['CO_MUNICIP', 'DS_NOME', 'CO_UF']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '431020')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '431020')
    how='left'                 # 'left' garante que não vamos perder nenhuma linha da principal
)

print("Merge com Municípios concluído!")

# Vamos verificar o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")

# Vamos renomear as colunas para ficar mais claro
df_analise.rename(columns={'DS_NOME': 'MUNICIPIO_PACIENTE', 'CO_UF': 'UF_PACIENTE'}, inplace=True)

# Mostra as colunas originais e as novas que foram "puxadas"
print(df_analise[['PA_MUNPCN', 'MUNICIPIO_PACIENTE', 'UF_PACIENTE']].head())

In [ ]:
# A chave da nossa tabela de análise é 'PA_PROC_ID'
chave_principal = 'PA_PROC_ID'

# A chave da nossa tabela auxiliar (procedimentos) é 'IP_COD'
chave_auxiliar = 'IP_COD'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_procedimentos[chave_auxiliar] = df_procedimentos[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

In [ ]:
# Agora fazemos o merge no 'df_analise' (que já tem os nomes dos municípios)
# Vamos puxar apenas a coluna 'IP_DSCR' (Descrição do Procedimento)
df_analise = pd.merge(
    left=df_analise,
    right=df_procedimentos[['IP_COD', 'IP_DSCR']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '0301010072')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '0301010072')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Procedimentos concluído!")

# Vamos renomear a nova coluna para ficar claro
df_analise.rename(columns={'IP_DSCR': 'NOME_PROCEDIMENTO'}, inplace=True)

# Verifica o resultado, mostrando a coluna do código e a nova coluna com o nome
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_PROC_ID', 'NOME_PROCEDIMENTO']].head())

In [ ]:
# A chave da nossa tabela de análise é 'PA_CODUNI'
chave_principal = 'PA_CODUNI'

# A chave da nossa tabela auxiliar (estabelecimentos) é 'CNES'
chave_auxiliar = 'CNES'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_estabelecimentos[chave_auxiliar] = df_estabelecimentos[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

In [ ]:
# Vamos puxar o 'NOME FANTASIA' e a 'RAZAO SOCIAL' do estabelecimento
df_analise = pd.merge(
    left=df_analise,
    right=df_estabelecimentos[['CNES', 'FANTASIA', 'RAZ_SOCI']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '2254611')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '2254611')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Estabelecimentos concluído!")

# Renomeia as colunas para ficar mais claro
df_analise.rename(columns={
    'FANTASIA': 'NOME_ESTABELECIMENTO',
    'RAZ_SOCI': 'RAZAO_SOCIAL_ESTAB'
}, inplace=True)

# Verifica o resultado, mostrando o código e os novos nomes
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CODUNI', 'NOME_ESTABELECIMENTO']].head())

In [ ]:
# A chave da nossa tabela de análise é 'PA_CIDPRI'
chave_principal = 'PA_CIDPRI'

# A chave da nossa tabela auxiliar (CID) é 'CD_COD'
chave_auxiliar = 'CD_COD'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_cid[chave_auxiliar] = df_cid[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

In [ ]:
# Vamos puxar a 'CD_DESCR' (Descrição do CID)
df_analise = pd.merge(
    left=df_analise,
    right=df_cid[['CD_COD', 'CD_DESCR']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: 'I10')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: 'I10')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Diagnósticos (CID) concluído!")

# Renomeia a coluna para ficar mais claro
df_analise.rename(columns={'CD_DESCR': 'NOME_DIAGNOSTICO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CIDPRI', 'NOME_DIAGNOSTICO']].head())

In [ ]:
# A chave da nossa tabela de análise é 'PA_CBOCOD'
chave_principal = 'PA_CBOCOD'

# A chave da nossa tabela auxiliar (CBO) é 'CBO'
chave_auxiliar = 'CBO'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_cbo[chave_auxiliar] = df_cbo[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

In [ ]:
# Vamos puxar a 'DS_CBO' (Descrição do CBO)
df_analise = pd.merge(
    left=df_analise,
    right=df_cbo[['CBO', 'DS_CBO']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '225125')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '225125')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Ocupações (CBO) concluído!")

# Renomeia a coluna para ficar mais claro
df_analise.rename(columns={'DS_CBO': 'NOME_OCUPACAO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CBOCOD', 'NOME_OCUPACAO']].head())

# Análise em ijuí

# Merge de tabelas apenas com os dados de "Ijuí"
Neste ponto reduzimos **13.2 milhões** de linhas para **160 mil registros** (os que realmente importam para Ijuí) essa ação é crucial para o bom andamento dos testes a seguir, melhorando muito nossa eficiência!

- O uso de memória caiu de **7.4+ GB** para apenas **86.7 MB**.
- Agora temos uma tabela leve, rápida e com todas as informações que precisamos para a análise

In [ ]:
import pandas as pd

print("--- Iniciando Análise Focada em Ijuí ---")

# --- PASSO 1: Encontrar o Código de Ijuí ---
# (Precisamos que a chave df_municipios['CO_MUNICIP'] já seja string)
try:
    ijui_info = df_municipios[df_municipios['DS_NOME'] == 'Ijui']
    codigo_ijui_str = ijui_info['CO_MUNICIP'].values[0]
    
    # A coluna PA_UFMUN (Município do Estabelecimento) é int64 (número)
    # Precisamos converter o código de Ijuí para int para o filtro funcionar
    codigo_ijui_int = int(codigo_ijui_str)
    
    print(f"Código de Ijuí encontrado: {codigo_ijui_int} (como número)")

except Exception as e:
    print(f"ERRO: Não foi possível encontrar o código de Ijuí. Verifique se df_municipios está carregado.")
    print(f"Detalhe do erro: {e}")

# --- PASSO 2: Filtrar a Tabela Gigante (13M de linhas) ---
print(f"Filtrando {len(df_producao_total)} registros para focar apenas em Ijuí...")

# Criamos um NOVO DataFrame SÓ com atendimentos feitos em estabelecimentos de Ijuí
# Usamos .copy() para garantir que é um objeto novo e evitar erros de memória
df_ijui_producao = df_producao_total[df_producao_total['PA_UFMUN'] == codigo_ijui_int].copy()

print(f"Filtro Concluído! Tabela reduzida para {len(df_ijui_producao)} registros.")

# --- PASSO 3: Preparar Chaves da Nova Tabela de Ijuí ---
# Agora, convertemos as chaves de merge *desta tabela pequena* para string
print("Preparando chaves da tabela de Ijuí para os merges...")
df_ijui_producao['PA_MUNPCN'] = df_ijui_producao['PA_MUNPCN'].astype(str)
df_ijui_producao['PA_PROC_ID'] = df_ijui_producao['PA_PROC_ID'].astype(str)
df_ijui_producao['PA_CODUNI'] = df_ijui_producao['PA_CODUNI'].astype(str)
df_ijui_producao['PA_CIDPRI'] = df_ijui_producao['PA_CIDPRI'].astype(str)
df_ijui_producao['PA_CBOCOD'] = df_ijui_producao['PA_CBOCOD'].astype(str)

# --- PASSO 4: Executar Merges na Tabela Pequena de Ijuí ---
print("Iniciando merges...")

# Merge 1: Município do Paciente (Onde o paciente mora)
df_analise = pd.merge(df_ijui_producao, df_municipios[['CO_MUNICIP', 'DS_NOME', 'CO_UF']], 
                      left_on='PA_MUNPCN', right_on='CO_MUNICIP', how='left')
df_analise.rename(columns={'DS_NOME': 'MUNICIPIO_PACIENTE', 'CO_UF': 'UF_PACIENTE'}, inplace=True)
print("Merge 1/5 (Município Paciente) OK")

# Merge 2: Nome do Procedimento
df_analise = pd.merge(df_analise, df_procedimentos[['IP_COD', 'IP_DSCR']], 
                      left_on='PA_PROC_ID', right_on='IP_COD', how='left')
df_analise.rename(columns={'IP_DSCR': 'NOME_PROCEDIMENTO'}, inplace=True)
print("Merge 2/5 (Nome Procedimento) OK")

# Merge 3: Nome do Estabelecimento (Hospital, UBS)
df_analise = pd.merge(df_analise, df_estabelecimentos[['CNES', 'FANTASIA']], 
                      left_on='PA_CODUNI', right_on='CNES', how='left')
df_analise.rename(columns={'FANTASIA': 'NOME_ESTABELECIMENTO'}, inplace=True)
print("Merge 3/5 (Nome Estabelecimento) OK")

# Merge 4: Nome do Diagnóstico (Doença)
df_analise = pd.merge(df_analise, df_cid[['CD_COD', 'CD_DESCR']], 
                      left_on='PA_CIDPRI', right_on='CD_COD', how='left')
df_analise.rename(columns={'CD_DESCR': 'NOME_DIAGNOSTICO'}, inplace=True)
print("Merge 4/5 (Nome Diagnóstico) OK")

# Merge 5: Nome da Ocupação (Profissional)
df_analise = pd.merge(df_analise, df_cbo[['CBO', 'DS_CBO']], 
                      left_on='PA_CBOCOD', right_on='CBO', how='left')
df_analise.rename(columns={'DS_CBO': 'NOME_OCUPACAO'}, inplace=True)
print("Merge 5/5 (Nome Ocupação) OK")

print("\n--- 💾 ANÁLISE DE IJUÍ PRONTA E CARREGADA ---")

# --- PASSO 5: Verificar Resultado ---
# Vamos ver o resultado e o novo uso de memória
df_analise.info()